<h1> Preparing the Data</h1>

This notebook will:
- import the master data
- expand the 'more_info' column to create a new df with all guitar details
- clean data
- perform feature engineering to numerically encode categorical data
- export model ready data


In [218]:
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings("ignore")

import ast
import re

from tqdm import tqdm
tqdm.pandas()

## Importing the Data

In [3]:
## import main data

#mvp_data = pd.read_csv('../data/guitar_scrap_02132022.csv')
#mvp_data = pd.read_csv('../data/eng_MVP_data.csv')
scrape_data = pd.read_csv('../data/full_scrape_02142022.csv')




scrape_data= scrape_data.drop(scrape_data.columns[[0]], axis=1) 
scrape_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 111303 entries, 0 to 111302
Data columns (total 5 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   title      111303 non-null  object
 1   price      111303 non-null  object
 2   condition  111303 non-null  object
 3   url        111303 non-null  object
 4   more_info  100000 non-null  object
dtypes: object(5)
memory usage: 4.2+ MB


In [5]:
scrape_data.head()

,title,price,condition,url,more_info
0,Warmoth Stratocaster (Mostly Loaded) Must See!,$950,Used – Very Good,https://reverb.com/item/45652241-warmoth-strat...,NaN
1,Fender American Professional II Stratocaster E...,"$1,599.99",Brand New,https://reverb.com/item/37165929-fender-americ...,{'title': 'Fender American Professional II Str...
2,"Eastman T486B-GB Gold Burst Deluxe 16"" Thinlin...","$1,319",Brand New,https://reverb.com/item/45652293-eastman-t486b...,{'title': 'Eastman T486B-GB Gold Burst Deluxe ...
3,Ibanez Artist 2618/12 string 1979 antique violin,"$2,499",Used – Very Good,https://reverb.com/item/45652250-ibanez-artist...,{'title': 'Ibanez Artist 2618/12 string 1979 a...
4,Pre-Owned 1988 Fender Made In Japan Stratocast...,"$1,299.99",Used – Very Good,https://reverb.com/item/45402369-pre-owned-198...,{'title': 'Pre-Owned 1988 Fender Made In Japan...


In [6]:
## import update to main data coming from latest web scrape
update_scrape = pd.read_csv('../data/update_scrape.csv')
update_scrape = update_scrape.drop(update_scrape.columns[[0,1,2]], axis=1) 
update_scrape.head()

,title,price,condition,url,more_info
0,Warmoth Stratocaster (Mostly Loaded) Must See!,$950,Used – Very Good,https://reverb.com/item/45652241-warmoth-strat...,NaN
1,Fender American Professional II Stratocaster E...,"$1,599.99",Brand New,https://reverb.com/item/37165929-fender-americ...,{'title': 'Fender American Professional II Str...
2,"Eastman T486B-GB Gold Burst Deluxe 16"" Thinlin...","$1,319",Brand New,https://reverb.com/item/45652293-eastman-t486b...,{'title': 'Eastman T486B-GB Gold Burst Deluxe ...
3,Ibanez Artist 2618/12 string 1979 antique violin,"$2,499",Used – Very Good,https://reverb.com/item/45652250-ibanez-artist...,{'title': 'Ibanez Artist 2618/12 string 1979 a...
4,Pre-Owned 1988 Fender Made In Japan Stratocast...,"$1,299.99",Used – Very Good,https://reverb.com/item/45402369-pre-owned-198...,{'title': 'Pre-Owned 1988 Fender Made In Japan...


In [7]:
all_df = pd.concat([scrape_data,update_scrape])
#all_df.drop_duplicates(keep=False,inplace=True) 
all_df = all_df[all_df.more_info.notnull()]
all_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 130000 entries, 1 to 70000
Data columns (total 5 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   title      130000 non-null  object
 1   price      130000 non-null  object
 2   condition  130000 non-null  object
 3   url        130000 non-null  object
 4   more_info  130000 non-null  object
dtypes: object(5)
memory usage: 6.0+ MB


### Expanding 'more_info'

In [221]:
def clean_year(txt):
    txt = txt.replace("'59",'1959').replace("'60",'1960').replace("'61",'1961').replace("'62",'1962')
    txt = txt.replace("'70s",'1970s')\
            .replace("'60s",'1960s')\
            .replace("'80s",'1980s')\
            .replace("'90s",'1990s')\
            .replace("'00s",'2000s')\
            .replace("'00s",'2000s')
    return txt


def expand_data(df):
    df = df[df.more_info.notnull()] # make sure there are no NaN in the more_info column
    
    project_data = df.copy().reset_index()
    dropped_count = 0 # counter to report if any records were dropped 
    
    expanded_project_data = pd.DataFrame()
    
    for val in tqdm(project_data.more_info):
        #print(val.index)
        val = val.replace('"',"'")
        val = clean_year(val)
        
        #line = re.sub(r"'\d\d", "", val)
        try:
            val = ast.literal_eval(val)
            more_df = pd.DataFrame([val])    
        
            #result = pd.concat([sampledata, more_df], axis=1, join='inner')
            expanded_project_data = expanded_project_data.append(more_df)
        except:
            dropped_count += 1
            #print('dropped record')
    print(f'There were {dropped_count} records that could not  be processed')
    return expanded_project_data


In [8]:
full_data = expand_data(all_df)

100%|█████████████████████████████████| 130000/130000 [1:10:02<00:00, 30.94it/s]

There were 17062 records that could not  be processed


In [10]:
full_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 112938 entries, 0 to 0
Data columns (total 19 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   title           112938 non-null  object
 1   brand           112938 non-null  object
 2   condition       112938 non-null  object
 3   categories      112938 non-null  object
 4   price           112938 non-null  object
 5   pickups         112938 non-null  object
 6   type            112938 non-null  object
 7   model           112938 non-null  object
 8   finish          112938 non-null  object
 9   origin          112938 non-null  object
 10  year            112938 non-null  object
 11  top             112938 non-null  object
 12  handed          112938 non-null  object
 13  neck            112938 non-null  object
 14  product_group   112938 non-null  object
 15  body_type       112938 non-null  object
 16  body_material   112938 non-null  object
 17  frets           112938 non-null  o

In [182]:
## save snaphot of data to csv
#full_data.to_csv('../data/full_data.csv', encoding='utf-8')
full_data = pd.read_csv('../data/full_data.csv',index_col=0)


In [183]:

full_data.head()

,title,brand,condition,categories,price,pickups,type,model,finish,origin,year,top,handed,neck,product_group,body_type,body_material,frets,available_sale
0,Fender American Professional II Stratocaster E...,Fender,Brand New,Solid Body,"1,799.99",SSS,Stratocaster,American Professional II Stratocaster with Map...,Natural Aaa,Asia,unknown,unknown,Right Handed,Maple,Fender Stratocaster,Solid Body,Pine,22,813
0,Ibanez Artist 2618/12 string 1979 antique violin,Ibanez,Used – Very Good,12-StringSolid Body,"2,499",custom,unknown,2618/12 AV Artist Series 12-String Double Cuta...,Antique Violin,Japan,1979,unknown,Right Handed,generic,generic,Solid Body,unknown,22,1
0,Pre-Owned 1988 Fender Made In Japan Stratocast...,Fender,Used – Very Good,unknown,"1,299.99",custom,unknown,Stratocaster,Olympic White,Japan,1988,unknown,Right Handed,generic,generic,generic,unknown,22,1
0,Ibanez Artcore AGS73T-TBC Semi-Hollow Body Ele...,Ibanez,Used – Excellent,Hollow Body,499.99,custom,unknown,AGS73T-TBC Artcore Expressionist Series Hollow...,unknown,Asia,unknown,unknown,Right Handed,generic,generic,Hollow Body,unknown,22,1
0,Strandberg Boden Standard 7,Strandberg,Brand New,Solid Body Electric Guitars,"1,695",custom,unknown,Boden Standard 7,BlackBlue,Indonesia,2018 - 2021,Maple,Right Handed,Maple,generic,Solid Body,unknown,22,1


## Inspect the data for feature engineering

In [184]:
full_data.describe()

,frets,available_sale
count,112938.000000,112938.000000
mean,22.025713,25.051161
std,0.461594,87.617393
min,21.000000,1.000000
25%,22.000000,1.000000
50%,22.000000,1.000000
75%,22.000000,2.000000
max,24.000000,813.000000


## Data Cleaning Tasks
------------------------------
* [price](#price)
    - clean and make price format float
    - remove rows with price unknown
    - remove ',' and make price format float
    

* [price range](#range)
    - create/encode pricing tiers for user to select in app    

* [origin](#origin)
    - clean up origin, some guitars are mislabeled with the wrong country
    - reduce and encode origin

* [type](#types)
    - encode type 
    
* [model](#model)
    - reduce and encode models
    
* [type](#clean)
    - final clean up tasks         


-------------------------------

## Fixing Price  <a class="anchor" id="price"></a>


In [185]:
# max price in df?
full_data.price.max()

'unknown'

In [186]:
# remove rows without price data
df_with_prices = full_data[full_data.price!='unknown']

In [187]:

def clean_price(txt):
    '''
    function to remove $ from price and replace commas then convert to float 
    '''        
    txt = txt.replace('$','')
    txt = txt.replace(',','')
    #txt = "{:.2f}".format(float(txt))
    #txt = float(txt)
    return float(txt)

In [188]:
## clean price by removing $ and commas and convert to float
df_with_prices['price'] = df_with_prices['price'].progress_apply(clean_price)

100%|███████████████████████████████| 102448/102448 [00:00<00:00, 772807.65it/s]


In [189]:
## Restrict our data to guitars 10k or less in value
df_under10k = df_with_prices[df_with_prices.price < 10000]
df_under10k = df_with_prices[df_with_prices.price > 300]
#df_under10k.info()

In [190]:
## lets look at how the prices are distributed
df_under10k.groupby(pd.cut(df_under10k["price"], np.arange(0, 10000, 500))).count()

,title,brand,condition,categories,price,pickups,type,model,finish,origin,year,top,handed,neck,product_group,body_type,body_material,frets,available_sale
price,,,,,,,,,,,,,,,,,,,
"(0, 500]",20264,20264,20264,20264,20264,20264,20264,20264,20264,20264,20264,20264,20264,20264,20264,20264,20264,20264,20264
"(500, 1000]",29929,29929,29929,29929,29929,29929,29929,29929,29929,29929,29929,29929,29929,29929,29929,29929,29929,29929,29929
"(1000, 1500]",14147,14147,14147,14147,14147,14147,14147,14147,14147,14147,14147,14147,14147,14147,14147,14147,14147,14147,14147
"(1500, 2000]",8672,8672,8672,8672,8672,8672,8672,8672,8672,8672,8672,8672,8672,8672,8672,8672,8672,8672,8672
"(2000, 2500]",5956,5956,5956,5956,5956,5956,5956,5956,4656,5956,5956,5956,5956,5956,5956,5956,5956,5956,5956
"(2500, 3000]",3797,3797,3797,3797,3797,3797,3797,3797,3797,3797,3797,3797,3797,3797,3797,3797,3797,3797,3797
"(3000, 3500]",1605,1605,1605,1605,1605,1605,1605,1605,1605,1605,1605,1605,1605,1605,1605,1605,1605,1605,1605
"(3500, 4000]",4032,4032,4032,4032,4032,4032,4032,4032,4032,4032,4032,4032,4032,4032,4032,4032,4032,4032,4032
"(4000, 4500]",3050,3050,3050,3050,3050,3050,3050,3050,3050,3050,3050,3050,3050,3050,3050,3050,3050,3050,3050


### Create price ranks <a class="anchor" id="origin"></a>
- max price is 6500 in our data. 
- ranges will be < 500, <1000, <1500, <2500, 2501 + 

In [191]:
def encode_price(df):
    conditions = [df.price < 500,
                  df.price < 1000,
                  df.price < 1500,
                  df.price < 2500,
                  df.price < 3500
                 ]
    
    values = [0,1,2,3,4]
    df['price_code'] = np.select(conditions, values, default=6)

In [192]:
encode_price(df_under10k)
df_under10k.price_code.value_counts()

1    29929
0    20264
2    14147
3    13992
6    12427
4     5842
Name: price_code, dtype: int64

-----------------------

## Cleaning up Origin <a class="anchor" id="origin"></a>

In [193]:
#full_data[(full_data['origin']=='Asia') & (full_data['brand']=='Ibanez')]
df_under10k[(df_under10k['origin']=='Asia')]

,title,brand,condition,categories,price,pickups,type,model,finish,origin,year,top,handed,neck,product_group,body_type,body_material,frets,available_sale,price_code
0,Fender American Professional II Stratocaster E...,Fender,Brand New,Solid Body,1799.99,SSS,Stratocaster,American Professional II Stratocaster with Map...,Natural Aaa,Asia,unknown,unknown,Right Handed,Maple,Fender Stratocaster,Solid Body,Pine,22,813,3
0,Ibanez Artcore AGS73T-TBC Semi-Hollow Body Ele...,Ibanez,Used – Excellent,Hollow Body,499.99,custom,unknown,AGS73T-TBC Artcore Expressionist Series Hollow...,unknown,Asia,unknown,unknown,Right Handed,generic,generic,Hollow Body,unknown,22,1,0
0,Ibanez RG1121PB RG Premium Electric Guitar - C...,Ibanez,Used – Mint,Solid Body,1169.99,custom,unknown,RG1121PB-CIF Premium,Caribbean Islet Flat,Asia,2020,unknown,Right Handed,generic,generic,Solid Body,unknown,22,27,2
0,Ibanez AT100CL-SB Andy Timmons Signature 6 Str...,Ibanez,Brand New,Solid Body,4170.11,custom,unknown,AT100CL-SB Andy Timmons Signature Prestige Ser...,unknown,Asia,unknown,unknown,Right Handed,generic,generic,Solid Body,unknown,22,2,6
0,Ibanez TQM1-NT Tom Quayle Signature,Ibanez,Brand New,Solid Body Electric Guitars,2599.99,custom,unknown,TQM1-NT Tom Quayle Signature with Roasted Mapl...,Natural,Asia,2010s,unknown,Right Handed,generic,generic,Solid Body,unknown,22,1,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,Sterling by Music Man Jason Richardson Signatu...,Sterling,Used – Mint,Solid Body,882.00,HH,unknown,Jason Richardson Signature 7-String Cutlass,Natural Poplar Burl Burst,Asia,unknown,unknown,Right Handed,Maple,generic,Solid Body,Alder,24,15,1
0,Ibanez RGDR4427FX Prestige,Ibanez,Brand New,Solid Body Electric Guitars,2399.99,HH,unknown,RGDR4427FX-NTF Prestige,Natural Flat,Asia,2021,unknown,Right Handed,generic,generic,Solid Body,Ash,24,1,3
0,Guyatone Zenon Teisco 1960 Sunburst Vintage G...,Guyatone,Good,Solid Body,380.40,custom,unknown,Zenon,Sunburst,Asia,1960,unknown,Right Handed,generic,generic,generic,unknown,22,1,0
0,Ibanez RGA42HPQM High Performance,Ibanez,Brand New,Solid Body Electric Guitars,899.99,HH,unknown,RGA42HPQM-BIG High Performance,Blue Iceburg Gradation,Asia,2021,Maple,Right Handed,Maple,generic,Solid Body,unknown,24,1,1


In [195]:
def clean_origin(df):    
    ## Correcting origin for American Made guitars
    
    df['origin'].loc[(df['model'].str.contains('American')) & (df['origin'] == 'Asia')] = 'United States'
    
    
    ## Correcting origin for Ibanez guitars
    df['origin'].loc[(df['origin']=='Asia') & 
                                     (df['model'].str.contains('AG')) &
                                     (df['brand']=='Ibanez')] = 'Japan'
    
    df['origin'].loc[(df['origin']=='Asia') & 
                                     (df['model'].str.contains('RG')) &
                                     (df['brand']=='Ibanez')] = 'Japan'
    
    df['origin'].loc[(df['origin']=='Asia') & 
                                     (df['model'].str.contains('RX')) &
                                     (df['brand']=='Ibanez')] = 'Japan'
    
    df['origin'].loc[(df['origin']=='Asia') & 
                                     (df['model'].str.contains('Prestige')) &
                                     (df['brand']=='Ibanez')] = 'Japan'
    
    df['origin'].loc[(df['origin']=='Asia') & 
                                     (df['model'].str.contains('Signature')) &
                                     (df['brand']=='Ibanez')] = 'Japan'
    
    df['origin'].loc[(df['origin']=='Asia') & 
                                     (df['model'].str.contains('Artcore')) &
                                     (df['brand']=='Ibanez')] = 'Korea'
    

## Clean up origin using function above

In [ ]:
clean_origin(df_under10k)

df_under10k.origin.value_counts()

## Reduce and Encode Origin

In [197]:
# feature origin code

def extract_origin(df):
    origin_dict = {
        'Asia':0,
        'China':0,
        'Russia':0,
        'Vietnam':0,
        'ChinaIndonesia':1,
        'Indonesia':1,
        'Vietnam':1,
        'Germany':2,
        'Korea':2,
        'Mexico':3,
        'Japan':3,
        'United States':4,
    }
    df['origin_code'] = (
        df.origin
          .str.extract('(' + '|'.join(origin_dict.keys()) + ')')
          .squeeze().map(origin_dict)
    )
    
    


In [198]:
extract_origin(df_under10k)
df_under10k = df_under10k[df_under10k['origin_code'].notna()]
df_under10k.head()

,title,brand,condition,categories,price,pickups,type,model,finish,origin,...,top,handed,neck,product_group,body_type,body_material,frets,available_sale,price_code,origin_code
0,Fender American Professional II Stratocaster E...,Fender,Brand New,Solid Body,1799.99,SSS,Stratocaster,American Professional II Stratocaster with Map...,Natural Aaa,United States,...,unknown,Right Handed,Maple,Fender Stratocaster,Solid Body,Pine,22,813,3,4.0
0,Ibanez Artist 2618/12 string 1979 antique violin,Ibanez,Used – Very Good,12-StringSolid Body,2499.00,custom,unknown,2618/12 AV Artist Series 12-String Double Cuta...,Antique Violin,Japan,...,unknown,Right Handed,generic,generic,Solid Body,unknown,22,1,3,3.0
0,Pre-Owned 1988 Fender Made In Japan Stratocast...,Fender,Used – Very Good,unknown,1299.99,custom,unknown,Stratocaster,Olympic White,Japan,...,unknown,Right Handed,generic,generic,generic,unknown,22,1,2,3.0
0,Ibanez Artcore AGS73T-TBC Semi-Hollow Body Ele...,Ibanez,Used – Excellent,Hollow Body,499.99,custom,unknown,AGS73T-TBC Artcore Expressionist Series Hollow...,unknown,Japan,...,unknown,Right Handed,generic,generic,Hollow Body,unknown,22,1,0,3.0
0,Strandberg Boden Standard 7,Strandberg,Brand New,Solid Body Electric Guitars,1695.00,custom,unknown,Boden Standard 7,BlackBlue,Indonesia,...,Maple,Right Handed,Maple,generic,Solid Body,unknown,22,1,3,1.0


## Encoding body_type <a class="anchor" id="type"></a>

In [199]:
# how many of each body type?
df_under10k.body_type.value_counts()

Solid Body          53372
generic             33406
Hollow Body          5484
Semi-hollow Body     4075
Name: body_type, dtype: int64

In [200]:
    
def encode_type(df):
    conditions = [df.type=='Solid Body',
                  df.type=='Semi-Hollow',
                  df.type=='Hollow Body',
                  df.type=='Other'
                 ]
    
    values = [3,2,1,0]
    df['type_code'] = np.select(conditions, values)

In [201]:
encode_type(df_under10k)
df_under10k.head()

,title,brand,condition,categories,price,pickups,type,model,finish,origin,...,handed,neck,product_group,body_type,body_material,frets,available_sale,price_code,origin_code,type_code
0,Fender American Professional II Stratocaster E...,Fender,Brand New,Solid Body,1799.99,SSS,Stratocaster,American Professional II Stratocaster with Map...,Natural Aaa,United States,...,Right Handed,Maple,Fender Stratocaster,Solid Body,Pine,22,813,3,4.0,0
0,Ibanez Artist 2618/12 string 1979 antique violin,Ibanez,Used – Very Good,12-StringSolid Body,2499.00,custom,unknown,2618/12 AV Artist Series 12-String Double Cuta...,Antique Violin,Japan,...,Right Handed,generic,generic,Solid Body,unknown,22,1,3,3.0,0
0,Pre-Owned 1988 Fender Made In Japan Stratocast...,Fender,Used – Very Good,unknown,1299.99,custom,unknown,Stratocaster,Olympic White,Japan,...,Right Handed,generic,generic,generic,unknown,22,1,2,3.0,0
0,Ibanez Artcore AGS73T-TBC Semi-Hollow Body Ele...,Ibanez,Used – Excellent,Hollow Body,499.99,custom,unknown,AGS73T-TBC Artcore Expressionist Series Hollow...,unknown,Japan,...,Right Handed,generic,generic,Hollow Body,unknown,22,1,0,3.0,0
0,Strandberg Boden Standard 7,Strandberg,Brand New,Solid Body Electric Guitars,1695.00,custom,unknown,Boden Standard 7,BlackBlue,Indonesia,...,Right Handed,Maple,generic,Solid Body,unknown,22,1,3,1.0,0


## Reducing and Encoding Condition <a class="anchor" id="condition"></a>

In [202]:
# encode Condition
def reduce_condition(df):
    conditions = [df.condition.str.contains('Mint'), 
                  df.condition.str.contains('Excellent'),
                  df.condition.str.contains('Very Good'),
                  df.condition.str.contains('Good'),
                  df.condition.str.contains('Fair'),
                  df.condition.str.contains('Poor')
                 ]
    
    values = ['Mint','Excellent','Very Good','Good','Fair','Poor']
    df['condition'] = np.select(conditions, values)
    
def encode_condition(df):
    conditions = [df.condition.str.contains('Mint'), 
                  df.condition.str.contains('Excellent'),
                  df.condition.str.contains('Very Good'),
                  df.condition.str.contains('Good'),
                  df.condition.str.contains('Fair'),
                  df.condition.str.contains('Poor')
                 ]
    
    values = [5,4,3,2,1,0]
    df['cond_score'] = np.select(conditions, values, default=3)
    

In [203]:
reduce_condition(df_under10k)
encode_condition(df_under10k)
df_under10k.head()

,title,brand,condition,categories,price,pickups,type,model,finish,origin,...,neck,product_group,body_type,body_material,frets,available_sale,price_code,origin_code,type_code,cond_score
0,Fender American Professional II Stratocaster E...,Fender,0,Solid Body,1799.99,SSS,Stratocaster,American Professional II Stratocaster with Map...,Natural Aaa,United States,...,Maple,Fender Stratocaster,Solid Body,Pine,22,813,3,4.0,0,3
0,Ibanez Artist 2618/12 string 1979 antique violin,Ibanez,Very Good,12-StringSolid Body,2499.00,custom,unknown,2618/12 AV Artist Series 12-String Double Cuta...,Antique Violin,Japan,...,generic,generic,Solid Body,unknown,22,1,3,3.0,0,3
0,Pre-Owned 1988 Fender Made In Japan Stratocast...,Fender,Very Good,unknown,1299.99,custom,unknown,Stratocaster,Olympic White,Japan,...,generic,generic,generic,unknown,22,1,2,3.0,0,3
0,Ibanez Artcore AGS73T-TBC Semi-Hollow Body Ele...,Ibanez,Excellent,Hollow Body,499.99,custom,unknown,AGS73T-TBC Artcore Expressionist Series Hollow...,unknown,Japan,...,generic,generic,Hollow Body,unknown,22,1,0,3.0,0,4
0,Strandberg Boden Standard 7,Strandberg,0,Solid Body Electric Guitars,1695.00,custom,unknown,Boden Standard 7,BlackBlue,Indonesia,...,Maple,generic,Solid Body,unknown,22,1,3,1.0,0,3


## Extracting/Encoding model/body style

In [204]:
df_under10k.type.value_counts()

unknown         84183
Stratocaster     7487
Telecaster       2442
Coronado          860
Mustang           652
Esquire           423
Cyclone           290
Name: type, dtype: int64

In [206]:
def clean_type(df): 
    ## cleaning up PRS types
    df['type'].loc[(df.type=='unknown') &
                            (df.brand=='PRS')]='PRS style'
    
    df['type'].loc[(df.type=='unknown') &
                            (df.title.str.contains('Silver Sky'))]='Stratocaster'
    
    #df_under10k['type'].loc[(df_under10k.type=='unknown') &
    #                        (df_under10k.title.str.contains('McCarty'))]='PRS style'
    
    
    ## capturing all Fender types
    df['type'].loc[(df.type=='unknown') &
                            (df.title.str.contains('Stratocaster'))]='Stratocaster'
    
    df['type'].loc[(df.type=='unknown') &
    df(df.title.str.contains('Tele'))]='Telecaster'
    
    df['type'].loc[(df.type=='unknown') &
                            (df.title.str.contains('Strat'))]='Stratocaster'
    
    df['type'].loc[(df.type=='unknown') &
                            (df.title.str.contains('Jazzmaster'))]='Jazzmaster'
    
    df['type'].loc[(df.type=='unknown') &
                            (df.title.str.contains('Jaguar'))]='Jaguar'
    
    ## capturing all ibanez types
    df['type'].loc[(df.type=='unknown') &
                            (df.title.str.contains('Ibanez'))]='Stratocaster'
    #df_under10k['type'].loc[(df_under10k.type=='unknown') &
    #                        (df_under10k.title.str.contains('Statocaster'))]='Strat'
    
    ## capturing all Gibson types
    df['type'].loc[(df.type=='unknown') &
                            (df.title.str.contains('Les Paul'))]='Les Paul'
    
    df['type'].loc[(df.type=='unknown') &
                            (df.title.str.contains('SG'))]='SG'
    
    df['type'].loc[(df.type=='unknown') &
                            (df.title.str.contains('Flying'))]='Flying V'
    
    df['type'].loc[(df.type=='unknown') &
                            (df.title.str.contains('ES'))]='ES'
    
    df['type'].loc[(df.type=='unknown') &
                            (df.title.str.contains('Casino'))]='ES'
    
    df['type'].loc[(df.type=='unknown') &
                        (df.title.str.contains('Sheraton'))]='ES'



## clean up type using function above

In [ ]:
clean_type(df_under10k)

In [209]:
df_under10k.type.value_counts()

unknown         38950
Stratocaster    24027
ES              12142
PRS style        5078
Telecaster       4944
Les Paul         4455
SG               3128
Jazzmaster       1388
Coronado          860
Mustang           652
Esquire           423
Cyclone           290
Name: type, dtype: int64

## Reduce and Encode types <a class="anchor" id="type"></a>

In [210]:
# feature engineer body_code 

def reduce_types(df): 
    body_dict = {
                'unknown':0,     
                'Stratocaster':4,
                'ES':4,          
                'PRS style':2,   
                'Telecaster':5,  
                'Les Paul':6,    
                'SG':3,          
                'Jazzmaster':1,  
                'Coronado':1,    
                'Mustang':1,           
                'Esquire':2,           
                'Cyclone':2,
                    }
    df['body_code'] = (
        df.type
          .str.extract('(' + '|'.join(body_dict.keys()) + ')')
          .squeeze().map(body_dict)
    )
    


In [211]:
reduce_types(df_under10k)
df_under10k.head()

,title,brand,condition,categories,price,pickups,type,model,finish,origin,...,product_group,body_type,body_material,frets,available_sale,price_code,origin_code,type_code,cond_score,body_code
0,Fender American Professional II Stratocaster E...,Fender,0,Solid Body,1799.99,SSS,Stratocaster,American Professional II Stratocaster with Map...,Natural Aaa,United States,...,Fender Stratocaster,Solid Body,Pine,22,813,3,4.0,0,3,4
0,Ibanez Artist 2618/12 string 1979 antique violin,Ibanez,Very Good,12-StringSolid Body,2499.00,custom,Stratocaster,2618/12 AV Artist Series 12-String Double Cuta...,Antique Violin,Japan,...,generic,Solid Body,unknown,22,1,3,3.0,0,3,4
0,Pre-Owned 1988 Fender Made In Japan Stratocast...,Fender,Very Good,unknown,1299.99,custom,Stratocaster,Stratocaster,Olympic White,Japan,...,generic,generic,unknown,22,1,2,3.0,0,3,4
0,Ibanez Artcore AGS73T-TBC Semi-Hollow Body Ele...,Ibanez,Excellent,Hollow Body,499.99,custom,Stratocaster,AGS73T-TBC Artcore Expressionist Series Hollow...,unknown,Japan,...,generic,Hollow Body,unknown,22,1,0,3.0,0,4,4
0,Strandberg Boden Standard 7,Strandberg,0,Solid Body Electric Guitars,1695.00,custom,unknown,Boden Standard 7,BlackBlue,Indonesia,...,generic,Solid Body,unknown,22,1,3,1.0,0,3,0


## some final clean up for outliers <a class="anchor" id="clean"></a>


In [215]:
df_under10k[df_under10k.cond_score>1]

,title,brand,condition,categories,price,pickups,type,model,finish,origin,...,product_group,body_type,body_material,frets,available_sale,price_code,origin_code,type_code,cond_score,body_code
0,Fender American Professional II Stratocaster E...,Fender,0,Solid Body,1799.99,SSS,Stratocaster,American Professional II Stratocaster with Map...,Natural Aaa,United States,...,Fender Stratocaster,Solid Body,Pine,22,813,3,4.0,0,3,4
0,Ibanez Artist 2618/12 string 1979 antique violin,Ibanez,Very Good,12-StringSolid Body,2499.00,custom,Stratocaster,2618/12 AV Artist Series 12-String Double Cuta...,Antique Violin,Japan,...,generic,Solid Body,unknown,22,1,3,3.0,0,3,4
0,Pre-Owned 1988 Fender Made In Japan Stratocast...,Fender,Very Good,unknown,1299.99,custom,Stratocaster,Stratocaster,Olympic White,Japan,...,generic,generic,unknown,22,1,2,3.0,0,3,4
0,Ibanez Artcore AGS73T-TBC Semi-Hollow Body Ele...,Ibanez,Excellent,Hollow Body,499.99,custom,Stratocaster,AGS73T-TBC Artcore Expressionist Series Hollow...,unknown,Japan,...,generic,Hollow Body,unknown,22,1,0,3.0,0,4,4
0,Strandberg Boden Standard 7,Strandberg,0,Solid Body Electric Guitars,1695.00,custom,unknown,Boden Standard 7,BlackBlue,Indonesia,...,generic,Solid Body,unknown,22,1,3,1.0,0,3,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,Ibanez RGDR4427FX Prestige,Ibanez,0,Solid Body Electric Guitars,2399.99,HH,Stratocaster,RGDR4427FX-NTF Prestige,Natural Flat,Japan,...,generic,Solid Body,Ash,24,1,3,3.0,0,3,4
0,Guyatone Zenon Teisco 1960 Sunburst Vintage G...,Guyatone,Good,Solid Body,380.40,custom,unknown,Zenon,Sunburst,Asia,...,generic,generic,unknown,22,1,0,0.0,0,2,0
0,Ibanez RGA42HPQM High Performance,Ibanez,0,Solid Body Electric Guitars,899.99,HH,Stratocaster,RGA42HPQM-BIG High Performance,Blue Iceburg Gradation,Japan,...,generic,Solid Body,unknown,24,1,1,3.0,0,3,4
0,Fender American Professional II Telecaster 3-C...,Fender,0,Solid Body,1699.99,SS,Telecaster,American Professional II Telecaster with Rosew...,3 Color Sunburst,United States,...,Fender Telecaster,Solid Body,Alder,22,393,3,4.0,0,3,5


## Export data for modeling <a class="anchor" id="model"></a>

In [212]:
modeling_df = df_under10k[['title',
                           'condition',
                           'model',
                           'brand',
                           'origin',
                           'type_code',
                           'origin_code',
                           'cond_score',
                           'body_code',
                           'price_code',
                           'price']].copy()
modeling_df.head()
                          

,title,condition,model,brand,origin,type_code,origin_code,cond_score,body_code,price_code,price
0,Fender American Professional II Stratocaster E...,0,American Professional II Stratocaster with Map...,Fender,United States,0,4.0,3,4,3,1799.99
0,Ibanez Artist 2618/12 string 1979 antique violin,Very Good,2618/12 AV Artist Series 12-String Double Cuta...,Ibanez,Japan,0,3.0,3,4,3,2499.00
0,Pre-Owned 1988 Fender Made In Japan Stratocast...,Very Good,Stratocaster,Fender,Japan,0,3.0,3,4,2,1299.99
0,Ibanez Artcore AGS73T-TBC Semi-Hollow Body Ele...,Excellent,AGS73T-TBC Artcore Expressionist Series Hollow...,Ibanez,Japan,0,3.0,4,4,0,499.99
0,Strandberg Boden Standard 7,0,Boden Standard 7,Strandberg,Indonesia,0,1.0,3,0,3,1695.00


In [213]:
modeling_df.to_csv('../data/modeling_df.csv', encoding='utf-8')

## Create a data cleaning function that will grab raw scraped data and clean it

In [ ]:
def clean_raw_data(df):
    '''
    This function will clean and prepare the webscraped data for modeling
    input: webscraped data as df
    output: clean, model-ready data
    '''
    # copy df first
    model_df = df.copy()
    expand(model_df)
    
    # remove data with no price
    model_df = model_df[model_df.price!='unknown']
    
    # Clean price 
    model_df['price'] = model_df['price'].progress_apply(clean_price)
    
    # limit data to >300 and <10000
    model_df = model_df[model_df.price < 10000]
    model_df = model_df[model_df.price > 300]
    
    # encode price
    encode_price(model_df)
    
    
    # extract origin and drop nulls
    clean_corigin(model_df)
    extract_origin(model_df)
    model_df = model_df[model_df['origin_code'].notna()]
    
    # reduce and encode condition
    reduce_condition(model_df)
    encode_condition(model_df)
    
    # clean up types
    clean_type(model_df)
    
    # reduce and encode type
    reduce_types(model_df)
    
    modeling_df = model_df[['title',
                         'condition',
                         'model',
                         'brand',
                         'origin',
                         'type_code',
                         'origin_code',
                         'cond_score',
                         'body_code',
                         'price_code',
                         'price']].copy()
    
    
    return modeling_df
    
    
    